# Experiment 4.5 – Real and Synthetic Data

## Setup

In [1]:
using Glob
using CSV
using DataFrames
using Distributions
using LinearAlgebra
using ProgressMeter

In [2]:
using Gadfly
import Cairo, Fontconfig

In [3]:
ENV["COLUMNS"] = 1000;

## Style

In [4]:
function get_match_ratio_colorid(x)
    if x == 1.0
        return 4
    elseif x >= 0.85
        return 3
    elseif x >= 0.5
        return 2
    else
        return 1
    end
end;

In [5]:
colors = [colorant"red", colorant"darkblue", colorant"deepskyblue", colorant"green"]

In [6]:
defaultfont = "CMU Serif";
fontstyles = (
    key_title_font = defaultfont,
    key_label_font = defaultfont,
    minor_label_font = defaultfont,
    major_label_font = defaultfont,
    key_title_font_size = 14px,
    key_label_font_size = 12px,
    minor_label_font_size = 14px,
    major_label_font_size = 16px,
);

## Load Data

In [7]:
fns = glob("*.csv", "../results/experiment_4_5a/");
results_list = [DataFrame(CSV.File(fn)) for fn in fns];
results_real = vcat(results_list...);

In [8]:
fns = glob("*.csv", "../results/experiment_4_5b/");
results_list = [DataFrame(CSV.File(fn)) for fn in fns];
results_syn = vcat(results_list...);

In [9]:
results_real_mean = combine(groupby(results_real, [:N, :ρe]), [
    :match_ratio => mean => :match_ratio,
    :alignment_strength_1 => mean => :alignment_strength_1,
]);
results_syn_mean = combine(groupby(results_syn, [:N, :ρe]), [
    :match_ratio => mean => :match_ratio,
    :alignment_strength_1 => mean => :alignment_strength_1,
]);

In [10]:
results_real.match_color = map(get_match_ratio_colorid, results_real.match_ratio);
results_real_mean.match_color = map(get_match_ratio_colorid, results_real_mean.match_ratio);

results_syn.match_color = map(get_match_ratio_colorid, results_syn.match_ratio);
results_syn_mean.match_color = map(get_match_ratio_colorid, results_syn_mean.match_ratio);

## Figures

In [11]:
figurespath = "./figures_45c/tex/"
if !isdir(figurespath) mkpath(figurespath) end;

In [12]:
function plot_fig1(df1, df2, v="v1"; display=true, save=false)
    @assert length(unique(df1.N)) == 1
    @assert length(unique(df2.N)) == 1
    
    if v == "v1"
        pointstyles = []
        figtype = "discretecolor_all"
    else
        pointstyles = (
            highlight_width=0px,
        )
        figtype = "discretecolor_mean"
    end
    if !isdir(joinpath(figurespath, figtype)) mkpath(joinpath(figurespath, figtype)) end
    
    _makeplt(_df, t) = plot(
        _df,
        x=:ρe, y=:alignment_strength_1, color=:match_color,
        Geom.point,

        Guide.xlabel("Parameter (ρ)"),
        Guide.ylabel("Alignment Strength"),
        Guide.title(t),

        Scale.color_discrete_manual(colors..., levels=[1,2,3,4]),
        Guide.colorkey(title="Match Ratio", labels=["&lt;50%", "≥50%", "≥85%", "100%"]),

        Guide.xticks(ticks=0:0.25:1),
        Guide.yticks(ticks=0:0.25:1),

        style(
            ;fontstyles...,
            pointstyles...,
        ),
    )
    
    plt1 = _makeplt(df1, "Real Data (n=$(df1.N[1]))")
    plt2 = _makeplt(df2, "Erdos-Renyi (n=$(df1.N[1]))")
    plt = hstack(plt1, plt2)
    
    if display
        plt |> SVG(34cm, 10cm)
    end
    if save
        n_str = lpad(df1.N[1]+1, 5, "0")
        plt |> PGF(joinpath(figurespath, "$(figtype)/exp45c_fig1_$(v)_$(n_str).tex"), 34cm, 10cm, texfonts=true)
    end
    
    return plt
end;

In [13]:
function plot_fig2(df1, df2, v="v1"; display=true, save=false)
    @assert length(unique(df1.N)) == 1
    @assert length(unique(df2.N)) == 1
    
    if v == "v1"
        pointstyles = []
        figtype = "continuouscolor_all"
    else
        pointstyles = (
            highlight_width=0px,
        )
        figtype = "continuouscolor_mean"
    end
    if !isdir(joinpath(figurespath, figtype)) mkpath(joinpath(figurespath, figtype)) end
    
    _makeplt(_df, t) = plot(
        _df,
        x=:ρe, y=:alignment_strength_1, color=:match_ratio,
        Geom.point,

        Guide.xlabel("Parameter (ρ)"),
        Guide.ylabel("Alignment Strength"),
        Guide.title(t),

        Guide.colorkey(title="Match Ratio"),

        Guide.xticks(ticks=0:0.25:1),
        Guide.yticks(ticks=0:0.25:1),

        style(
            ;fontstyles...,
            pointstyles...,
        ),
    )
    
    plt1 = _makeplt(df1, "Real Data (n=$(df1.N[1]))")
    plt2 = _makeplt(df2, "Erdos-Renyi (n=$(df1.N[1]))")
    plt = hstack(plt1, plt2)
    
    if display
        plt |> SVG(34cm, 10cm)
    end
    if save
        n_str = lpad(df1.N[1]+1, 5, "0")
        plt |> PGF(joinpath(figurespath, "$(figtype)/exp45c_fig2_$(v)_$(n_str).tex"), 34cm, 10cm, texfonts=true)
    end
    
    return plt
end;

In [14]:
for N in sort(unique(results_real.N))
    @show N
    
    r_real = filter(row -> row.N == N, results_real)
    r_real_mean = filter(row -> row.N == N, results_real_mean)
    
    r_syn = filter(row -> row.N == N, results_syn)
    r_syn_mean = filter(row -> row.N == N, results_syn_mean)
    
    if nrow(r_syn) == 0
        println("skipped")
        continue
    end
    
    plot_fig1(r_real, r_syn, "v1", display=false, save=true)
    plot_fig1(r_real_mean, r_syn_mean, "v2", display=false, save=true)
    plot_fig2(r_real, r_syn, "v1", display=false, save=true)
    plot_fig2(r_real_mean, r_syn_mean, "v2", display=false, save=true)
end

N = 70
N = 94
N = 107
N = 139
N = 194
N = 277
N = 349
N = 445
N = 582
N = 832
N = 1215
N = 1874
N = 3230
